# Water Quality Parameters Maps - Production

Import the necessary packages to run the processing of the files.

In [1]:
# Styling notebook

# System
import os
import sys

# Import scripts libraries for the project
sys.path.append('./src/python')

# Import the function to update the notebook style
from nbConfig import (css_styling)

css_styling()

## 0. Import WQP maps production library

Import the SNAP-py library and function parameters files for the processing of the WQPs maps.

In [2]:
import pandas as pd
import ipywidgets as wg
import wqpSNAPparams_S3 as wqpParams_S3
import wqpSNAPparams_L8 as wqpParams_L8
import wqpSNAPparams_EUMETSAT as wqpParams_EUMETSAT
import wqpSNAPFunctions as wqpSNAP

Select the collection of interest for the processing of the data.

**Note:** Make sure to have uploaded the files of the satellite imagery in the respective folder for the processing.

- Sentinel-3 A/B: `'./in/data/satellite_imagery/S3'`
- Landsat-8: `'./in/data/satellite_imagery/L8'`
- EUMETSAT: `'./in/data/satellite_imagery/EUMETSAT'`

Each product must be uncompressed and stored in an independent folder

In [3]:
SENSOR_LIST = ['S3','L8','EUMETSAT']
sensor = wg.Select(
    options = SENSOR_LIST,
    value = SENSOR_LIST[0],
    description = 'Select the sensor:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='50%', height='100px'),
)

In [4]:
sensor

Select(description='Select the sensor:', layout=Layout(height='100px', width='50%'), options=('S3', 'L8', 'EUM…

In [5]:
in_path = f'./in/satellite_imagery/{sensor.value}'
out_path = f'./in/wqp/{sensor.value}'
cwd_path = wqpSNAP.inputParameters(in_path,out_path)

In [6]:
cwd_path

{'in': './in/satellite_imagery/S3',
 'out': './in/wqp/S3',
 'out_masks': './in/wqp/S3/masks',
 'out_oa': './in/wqp/S3/oa',
 'out_rrs': './in/wqp/S3/rrs',
 'out_wqp': './in/wqp/S3/wqp',
 'out_wqp_no_clip': './in/wqp/S3/wqp_no_clip',
 'out_wqp_cloud': './in/wqp/S3/wqp_cloud_mask',
 'out_wqp_no_mask': './in/wqp/S3/wqp_no_mask',
 'in_parameters': './in/satellite_imagery/wqp_parameters',
 'vectorFile': './vector/simile_laghi/simile_laghi.shp'}

## 1. Define processing parameters

Import the additional input parameters used in the processing of the data. For the processing of CHL-a and TSM products using the Sentinel-3 OLCI sensor, it is required the mean temperature in the region of interest. While, for the processing of the LSWT products using the Landsat-8 TIRS sensor, are required some atmospheric correction parameters presented in the [Barsi method](https://www.mdpi.com/2072-4292/6/11/11607) (Barsi, 2014)

### 1.1. Temperature Data - ARPA Lombardia

The temperature data is retrieved from the `./in/satellite_imagery/wqp_parameters/meteoTemp.csv` file. Make sure to update the temperature measurements to match the satellite images acquisition timestamps to allow the processing.

The update of the temperature measurements is possible either by following the procedure presented in the [downloadMeteoARPA](./downloadMeteoARPA.ipynb) notebook, or by requesting the the data from [ARPA Lombardia](https://www.arpalombardia.it/Pages/Meteorologia/Richiesta-dati-misurati.aspx#) by manually updating the **.csv** file.

In [7]:
#Read the mean temperature file
df_t = pd.read_csv(os.path.join(cwd_path['in_parameters'],'meteoTemp.csv'))
df_t["Data"] = pd.to_datetime(df_t['Data'])
df_t_keys = list(df_t.keys())

In [8]:
# Sample of the temperature dataset
df_t.head()

,IdSensore,Data,Valore,Stato,idOperatore
0,14606,2019-01-01 00:00:00,5.5,VA,1
1,14606,2019-01-01 00:10:00,5.5,VA,1
2,14606,2019-01-01 00:20:00,5.4,VA,1
3,14606,2019-01-01 00:30:00,5.3,VA,1
4,14606,2019-01-01 00:40:00,5.3,VA,1


### 1.2. Atmospheric Correction Parameters

The atmospheric correction parameters are manually updated by requesting them at the corresponding time of acquisition of the Landsat images. The parameters are computed using the [NASA Atmospheric Correction Calculator](https://atmcorr.gsfc.nasa.gov/atm_corr.html), and are appended into the `./in/satellite_imagery/wqp_parameters/atmCorr.csv` file. 

Here, we will make an example for the Landsat Image "*LC08_L1GT_194028_20220506_20220512_02_T2*", where the date of acquisition corresponds to the 4th string (2022-05-06). The time of acquisition can be found inside the metadata file for the products **'*_MTL.xml'** file. For this products the time is around 10:10 at the center time of the scene. The requests would be carried out as follows:

<div style="display: inline-flex">
    <div style="margin: 10px; width:45%">
        <h5>Atmospheric Correction Parameters Request Module</h5>
        <img src='./src/images/sample_form_atmCorr.png' style='width:95%'>
    </div> 
    <div style="margin: 10px; width:45%">
        <h5>Atmospheric Correction Parameters Request Output</h5>
        <img src='./src/images/sample_output_atmCorr.png' style='width:95%'>
    </div> 
</div>

Once the parameters are obtained, it is possible to update the **atmCorr.csv** file with the help of a text editor, or excel, to add a ne line for the new record. Notice that the data must be compiled in the same order as stated in the first line.

<div style="display: flex; align-items: center; justify-content: center">
    <div style="margin: 10px">
        <h5>Atmospheric Correction Parameters File Update</h5>
        <img src='./src/images/sample_update_atmCorr.png' style='width:40%'>
    </div> 
</div>

In [9]:
# Read the atmospheric correction parameters file
df_atm = pd.read_csv(os.path.join(cwd_path['in_parameters'],'atmCorr.csv'))
df_atm_keys = list(df_t.keys())

In [10]:
df_atm['DateTime'] = pd.to_datetime(df_atm['DateTime'], format='%Y/%m/%d %H:%M')
df_atm['DateTime'] = df_atm['DateTime'].dt.date

In [11]:
df_atm.head()

,DateTime,Year,Month,Day,GMT Hour,Minute,t,Lu,Ld
0,2019-01-01,2019,1,1,10,10,0.93,0.47,0.80
1,2019-01-06,2019,1,6,10,10,0.93,0.46,0.79
2,2019-01-13,2019,1,13,10,10,0.91,0.56,0.96
3,2019-01-15,2019,1,15,10,10,0.95,0.32,0.55
4,2019-01-17,2019,1,17,10,10,0.87,0.80,1.34


## 2. WQP maps production

This section presents the processing of the Water Quality Parameters maps correspongding to the specific product and map typology.

In [12]:
# Temptative bounding box for the area of interest
bbox = {
    'minLat' : 45.3,
    'maxLat' : 46.65,
    'minLon' : 7.9,
    'maxLon' : 9.95,
}

In [13]:
# Format output processed images
writeFormat = 'GeoTIFF'

### S3 - Chlorophyll-a (CHL-a) and Total Suspended Matter (TSM)

Here, the WQP maps using the Sentinel-3A/B images are derived. The retrieval of the sentinel images can be achieved through the procedure described in the notebook [downloadWEkEO](./downloadWEkEO.ipynb) to import the images directly into the processing directory. Alternatively, it is possible to download the images through [Copernicus Access Hub](https://scihub.copernicus.eu/dhus/#/home), add move the images into the `./in/satellite_imagery/S3` folder and unzip the products into separate folders. 

In [15]:
wqpParams = wqpParams_S3
for root, dirs, files in os.walk(in_path):
    for d in dirs:
        if (d != '.ipynb_checkpoints'):
            # 1. Read the product
            s3_image = wqpSNAP.snapProduct(os.path.join(root,d,'xfdumanifest.xml'),bbox)
            s3_image.readSNAPProduct()
            s3_image.name = s3_image.path.split('/')[-2]
            print(s3_image.name)
            try:
                # 2. Update the bounding box for the subset selection
                params_subset = s3_image.updateSNAPSubset(wqpParams.params_subset)
                subset_product = wqpSNAP.executeSNAPFunction(s3_image.product, params_subset)
                # 3. Reproject the subset
                reproject_product = wqpSNAP.executeSNAPFunction(subset_product, wqpParams.params_reproject)
                # 4. Update the C2RCC temperature value. C2RCC wqp products
                params_C2RCC = s3_image.updateSNAPTemperature(df_t, wqpParams.params_C2RCC)
                c2rcc_product = wqpSNAP.executeSNAPFunction(reproject_product, params_C2RCC)
                # 5. Import vector layer
                importVector_product = wqpSNAP.executeSNAPFunction(c2rcc_product, wqpParams.params_importVector)
                # 6. Band Maths operations
                bandMaths_product_C2RCC = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths)
                bandMaths_product_oa = wqpSNAP.executeSNAPFunction(reproject_product, wqpParams.params_bandMaths_oa)
                # bandMaths_product_rrs = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths_rrs)  
                # bandMaths_product_masks = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths_masks)
                # 7. Extract bands
                # Product bands to be extracted
                bandExtract_product_chl = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_chl)
                bandExtract_product_tsm = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_tsm)
                # bandExtract_product_chl_no_clip = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_chl_no_clip)
                # bandExtract_product_tsm_no_clip = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_tsm_no_clip)
                # bandExtract_product_chl_no_mask = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_chl_no_masks)
                # bandExtract_product_tsm_no_mask = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_tsm_no_masks)
                # bandExtract_product_chl_cloud_mask = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_chl_cloud_mask)
                # bandExtract_product_tsm_cloud_mask = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_tsm_cloud_mask)
                # bandExtract_product_oa = wqpSNAP.executeSNAPFunction(bandMaths_product_oa, wqpParams.params_bandExtractor_oa)
                bandExtract_product_oa = wqpSNAP.executeSNAPFunction(bandMaths_product_oa, wqpParams.params_bandExtractor_oa_18)
                # bandExtract_product_rrs = wqpSNAP.executeSNAPFunction(bandMaths_product_rrs, wqpParams.params_bandExtractor_rrs)
                # bandExtract_product_masks = wqpSNAP.executeSNAPFunction(bandMaths_product_masks, wqpParams.params_bandExtractor_masks)
                # 8. Save bands
                # Define output paths
                sensorName = s3_image.name.split('_')[0]
                sensorDate = s3_image.name.split('_')[8]
                print(sensorName, sensorDate)
                out_path_chl = os.path.join(cwd_path['out_wqp']+'/chl/'+sensorName+'_CHL_IT_'+sensorDate+'_L1')
                out_path_tsm = os.path.join(cwd_path['out_wqp']+'/tsm/'+sensorName+'_TSM_IT_'+sensorDate+'_L1')
                # out_path_chl_no_clip = os.path.join(cwd_path['out_wqp_no_clip']+'\\chl\\'+sensorName+'_CHL_IT_'+sensorDate+'_L1')
                # out_path_tsm_no_clip = os.path.join(cwd_path['out_wqp_no_clip']+'\\tsm\\'+sensorName+'_TSM_IT_'+sensorDate+'_L1')
                # out_path_chl_no_mask = os.path.join(cwd_path['out_wqp_no_mask']+'\\chl\\'+sensorName+'_CHL_IT_'+sensorDate+'_L1')
                # out_path_tsm_no_mask = os.path.join(cwd_path['out_wqp_no_mask']+'\\tsm\\'+sensorName+'_TSM_IT_'+sensorDate+'_L1')
                # out_path_chl_cloud_mask = os.path.join(cwd_path['out_wqp_cloud']+'\\chl\\'+sensorName+'_CHL_IT_'+sensorDate+'_L1')
                # out_path_tsm_cloud_mask = os.path.join(cwd_path['out_wqp_cloud']+'\\tsm\\'+sensorName+'_TSM_IT_'+sensorDate+'_L1')
                # out_path_mask = os.path.join(cwd_path['out_masks']+'\\'+sensorName+'_IT_'+sensor_date+'_L1')
                out_path_oa = os.path.join(cwd_path['out_oa']+'\\'+sensorName+'_IT_'+sensorDate+'_L1')
                # out_path_rrs = os.path.join(cwd_path['out_rrs']+'\\'+sensorName+'_IT_'+sensorDate+'_L1')
                # Save Bands
                wqpSNAP.exportProductBands(bandExtract_product_chl, out_path_chl, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_tsm, out_path_tsm, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_chl_no_clip, out_path_chl_no_clip, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_tsm_no_clip, out_path_tsm_no_clip, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_chl_no_mask, out_path_chl_no_mask, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_tsm_no_mask, out_path_tsm_no_mask, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_chl_cloud_mask, out_path_chl_cloud_mask, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_tsm_cloud_mask, out_path_tsm_cloud_mask, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_masks, out_path_mask, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_oa, out_path_oa, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_rrs, out_path_rrs, writeFormat)
                # Clean environment
                subset_product.dispose() 
                reproject_product.dispose()
                c2rcc_product.dispose()
                importVector_product.dispose()
                bandExtract_product_chl.dispose()
                bandExtract_product_tsm.dispose()
                # bandExtract_product_chl_no_mask.dispose()
                # bandExtract_product_tsm_no_mask.dispose()
                # bandExtract_product_chl_cloud_mask.dispose()
                # bandExtract_product_tsm_cloud_mask.dispose()
                # bandExtract_product_rrs.dispose()
                # bandExtract_product_oa.dispose()
                # bandExtract_product_masks.dispose()
                del s3_image
                del subset_product
                del reproject_product
                del c2rcc_product
                del importVector_product
                del bandExtract_product_chl
                del bandExtract_product_tsm
                # del bandExtract_product_chl_no_mask
                # del bandExtract_product_tsm_no_mask
                # del bandExtract_product_chl_cloud_mask
                # del bandExtract_product_tsm_cloud_mask
                # del bandExtract_product_rrs
                # del bandExtract_product_oa
                # del bandExtract_product_masks
            except:
                # Open a file with access mode 'a'
                file_object = open(os.path.join(cwd_path['out'],f'error_images_{sensor.value}.txt'), 'a')
                # Append 'hello' at the end of file
                file_object.write(s3_image.name)
                file_object.write("\n")
                # Close the file
                file_object.close()

S3A_OL_1_EFR____20220907T100157_20220907T100457_20220908T153607_0179_089_293_2160_MAR_O_NT_002.SEN3
Date: 2022-09-07 10:00 
Temperature: 21.2°C
bandmaths
S3A_OL_1_EFR____20220908T093546_20220908T093846_20220909T151856_0179_089_307_2160_MAR_O_NT_002.SEN3
Date: 2022-09-08 09:30 
Temperature: 18.3°C
bandmaths
S3A_OL_1_EFR____20220909T090935_20220909T091235_20220910T135824_0179_089_321_2160_MAR_O_NT_002.SEN3
Date: 2022-09-09 09:00 
Temperature: 20.7°C
bandmaths
S3A_OL_1_EFR____20220911T095813_20220911T100113_20220912T142011_0179_089_350_2160_MAR_O_NT_002.SEN3
Date: 2022-09-11 09:50 
Temperature: 23.0°C
bandmaths
S3A_OL_1_EFR____20220912T093202_20220912T093502_20220913T155208_0179_089_364_2160_MAR_O_NT_002.SEN3
Date: 2022-09-12 09:30 
Temperature: 22.3°C
bandmaths
S3A_OL_1_EFR____20220915T095429_20220915T095729_20220916T163444_0179_090_022_2160_MAR_O_NT_002.SEN3
Date: 2022-09-15 09:50 
Temperature: 25.2°C
bandmaths
S3A_OL_1_EFR____20220916T092818_20220916T093118_20220917T140322_0179_090_036

KeyboardInterrupt: 

**Note:** There are several products that you can extract from the outputs of the WQP processing. Depending on your needs you can uncomment the different lines in the code to produce them. Here, we focus in the production of the WQP maps and the extraction of the reference bands used for the coregistration of the images. In this case, the reference band used for the images coregistration of the images correspond to Band 18 of the S3-OLCI images ([Sentinel-3 OLCI bands reference](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-3-olci/resolutions/radiometric)).

### L8 - Land Surface Water Temperature (LSWT)

Here, the WQP maps using the Landsat-8 images are derived. It is possible to download the images through [USGS Earth explorer](https://earthexplorer.usgs.gov/), add move the images into the `./in/satellite_imagery/L8` folder and unzip the products into separate folders.

In [ ]:
wqpParams = wqpParams_L8
for root, dirs, files in os.walk(in_path):
    for f in files:
        if f.endswith('MTL.txt'):
            print(os.path.join(root,f),bbox)
            # 1. Read the product
            l8_image = wqpSNAP.snapProduct(os.path.join(root,f),bbox)
            l8_image.readSNAPProduct()
            l8_image.name = l8_image.path.split('/')[-2].split('.')[0]
            params_bandMaths = l8_image.updateSNAPAtmCorr(df_atm,wqpParams.params_bandMaths)
            try:
                # 2. Update the bounding box for the subset selection
                params_subset = l8_image.updateSNAPSubset(wqpParams.params_subset)
                subset_product = wqpSNAP.executeSNAPFunction(l8_image.product, params_subset)
                # 3. Reproject the subset
    #                 reproject_product = wqpSNAP.executeSNAPFunction(subset_product, wqpParams.params_reproject) 
                # 4. Resample
                resample_product = wqpSNAP.executeSNAPFunction(subset_product, wqpParams.params_resample)
                # 5. Import-Vector
                importVector_product = wqpSNAP.executeSNAPFunction(resample_product, wqpParams.params_importVector)
                # 6. BandMaths
                bandMaths_product = wqpSNAP.executeSNAPFunction(importVector_product, params_bandMaths)
                # bandMaths_product_masks = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths_masks)
                # 7. BandExtract
                # bandExtract_product_lswt = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_lswt)
                # bandExtract_product_lswt_mid_high = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_lswt_mid_high)
                bandExtract_product_lswt_high = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_lswt_high)
                # bandExtract_product_masks = wqpSNAP.executeSNAPFunction(bandMaths_product_masks, wqpParams.params_bandExtractor_masks)
                # 8. Export bands
                # Output paths
                sensor_date = l8_image.name.split('_')[3]
                # out_path_lswt = os.path.join(out_path,'wqp','lswt',sensor.value+'_LSWT_IT_'+sensor_date+'_L1')
                # out_path_lswt_mid_high = os.path.join(out_path,'wqp_mid_high_clouds','lswt',sensor.value+'_LSWT_IT_'+sensor_date+'_L1')
                out_path_lswt_high = os.path.join(out_path,'wqp','lswt',sensor.value+'_LSWT_IT_'+sensor_date+'_L1')
                # out_path_mask = os.path.join(out_path,'masks',sensor.value+'_LSWT_IT_'+sensor_date+'_L1')
                #Save Bands
                # wqpSNAP.exportProductBands(bandExtract_product_lswt, out_path_lswt, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_lswt_mid_high, out_path_lswt_mid_high, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_lswt_high, out_path_lswt_high, writeFormat)
                # wqpSNAP.exportProductBands(bandExtract_product_masks, out_path_mask, writeFormat)
                
                subset_product.dispose() 
                importVector_product.dispose()
                resample_product.dispose()
                bandMaths_product.dispose()
                bandMaths_product_masks.dispose()
                bandExtract_product_lswt.dispose()
                bandExtract_product_lswt_mid_high.dispose()
                bandExtract_product_lswt_high.dispose()
                bandExtract_product_masks.dispose()
#                 del l8_image
                del subset_product
                del importVector_product
                del resample_product
                # del bandExtract_product_lswt
                # del bandExtract_product_lswt_mid_high
                del bandExtract_product_lswt_high
                # del bandExtract_product_masks
                del bandMaths_product
                # del bandMaths_product_masks
            except:
                # Open a file with access mode 'a'
                file_object = open(os.path.join(cwd_path['out'],f'error_images_{sensor.value}.txt'), 'a')
                file_object.write(l8_image.name)
                file_object.write("\n")
                # Close the file
                file_object.close()

### EUMETSAT products extract

In [ ]:
wqpParams = wqpParams_EUMETSAT
print(in_path)
#Image processing of the files in the input folder
for root, directories, files in os.walk(in_path, topdown=False):
    for f in files:
        if (f.endswith('.xml')):
            try:
                # 1. Read the product
                EUMETSAT_image = wqpSNAP.snapProduct(os.path.join(root,f),bbox)
                EUMETSAT_image.readSNAPProduct()
                EUMETSAT_image.name = EUMETSAT_image.path.split('\\')[-2].split('.')[0]
                print(EUMETSAT_image.name)
                # 2. Update the bounding box for the subset selection
                params_subset = EUMETSAT_image.updateSNAPSubset(wqpParams.params_subset)
                subset_product = wqpSNAP.executeSNAPFunction(EUMETSAT_image.product, params_subset)
                # 3. Reproject
                reproject_product = wqpSNAP.executeSNAPFunction(subset_product, wqpParams.params_reproject)
                # 4. Import-Vector
                importVector_product = wqpSNAP.executeSNAPFunction(reproject_product, wqpParams.params_importVector)
                # 5. BandMaths
                bandMaths_product = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths)
                # 6. BandExtract+
                bandExtract_product_chl_nn = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_chl_nn)
                bandExtract_product_chl_oc4me = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_chl_oc4me)
                bandExtract_product_tsm_nn = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_tsm_nn)
                # 7. Write Subset test
                sensor_date = EUMETSAT_image.name.split('_')[7]
                sensor_value = EUMETSAT_image.name.split('_')[0]
                out_path_chl_nn = os.path.join(out_path,'out_chl_nn')
                out_chl_oc4me = os.path.join(out_path,'out_chl_oc4me')
#                 out_tsm_nn = os.path.join(out_path,'out_tsm_nn')
                wqpSNAP.exportProductBands(bandExtract_product_chl_nn ,os.path.join(out_path_chl_nn,sensor_value+'_CHL_IT_'+sensor_date+'_L1'), writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_chl_oc4me,os.path.join(out_chl_oc4me,sensor_value+'_CHL_IT_'+sensor_date+'_L1'), writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_tsm_nn,os.path.join(out_tsm_nn,sensor_value+'_TSM_IT_'+sensor_date+'_L1'), writeFormat)
                
                subset_product.dispose() 
                importVector_product.dispose()
                bandMaths_product.dispose()
                bandExtract_product_chl_nn.dispose()
                bandExtract_product_chl_oc4me.dispose()
                bandExtract_product_tsm_nn.dispose()
#                 del l8_image
                del subset_product
                del importVector_product
                del bandExtract_product_chl_nn
                del bandExtract_product_chl_oc4me
                del bandExtract_product_tsm_nn
            except:
                # Open a file with access mode 'a'
                file_object = open(os.path.join('./in/data/satellite_imagery',f'error_images_{sensor.value}.txt'), 'a')
                # Append 'hello' at the end of file
                file_object.write(EUMETSAT_image.name)
                file_object.write("\n")
                # Close the file
                file_object.close()